# [ 프로젝트 - Movielens 영화 추천 실습 ]

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

    * 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
    * 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
    * 별점을 시청횟수로 해석해서 생각하겠습니다.
    * 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

1) wget으로 데이터 다운로드 <br/>
    $ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 이동 <br/>
    $ mv ml-1m.zip ~/workplace/recommendation_movie/data

3) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다. <br/>
    $ unzip ml-1m.zip

In [2]:
import pandas as pd
import numpy as np
import os

## 1. 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [7]:
rating_file_path = os.getenv('HOME')+'/workplace/recommendation_movie/data/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp'] # rating: 별점(시청횟수로 간주)
ratings = pd.read_csv(rating_file_path, sep='::', names = ratings_cols, engine='python')
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
# 3점 이상만 남김: ratings = ratings[ratings['rating']>=3]

ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is { filtered_data_size / original_data_size: }')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is  0.8363032126285607


In [12]:
# rating컬럼의 이름을 count로 바꿈: rename 사용!!

ratings.rename(columns={'rating':'count'}, inplace=True)

In [13]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [32]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.

movie_file_path = os.getenv('HOME')+'/workplace/recommendation_movie/data/movies.dat'
cols = ['movie_id','title','genre']   # genre: 장르
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2. 분석해 봅시다.

   * ratings에 있는 유니크한 영화 개수
   * rating에 있는 유니크한 사용자 수
   * 가장 인기있는 영화 30개(인기순)


In [17]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [29]:
movies['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Tigerland (2000)', 'Two Family House (2000)',
       'Contender, The (2000)'], dtype=object)

In [18]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [27]:
# 가장 인기있는 영화 30개(인기순)

ratings_count = ratings.groupby('movie_id')['user_id'].count()
ratings_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [41]:
tmp = movies['title'].isin(['Intern','La La Land', 'Harry Potter', 'Interstellar', 'The Holiday'])
tmp

0       False
1       False
2       False
3       False
4       False
        ...  
3878    False
3879    False
3880    False
3881    False
3882    False
Name: title, Length: 3883, dtype: bool

In [39]:
my_favorite = ['Intern','La La Land', 'Harry Potter', 'Interstellar', 'The Holiday' ]

In [ ]:
if not ratings.isin({''})

## 4. CSR matrix를 직접 만들어 봅시다.

## 5. als_model = AlternatingLeastSquares 

## 모델을 직접 구성하여 훈련시켜 봅시다.

## 6. 내가 선호하는 5가지 영화 중 하나와 

## 그 외의 영화 하나를 골라 훈련된 모델이 예측한 

## 나의 선호도를 파악해 보세요.

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.